In [30]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from elasticsearch import Elasticsearch

es = Elasticsearch('http://192.168.2.23:9200')
elasticsearch_index = "aircraft_data"

def get_data(time_window_hours=24, num_samples=1000):
    # documents has 'created_on' field as seconds elapsed since epoch
    start_time = int((datetime.datetime.now() - datetime.timedelta(hours=time_window_hours)).timestamp())
    end_time = int(datetime.datetime.now().timestamp())

    query = {
        "query": {
            "range": {
                "created_on": {
                    "gte": start_time,
                    "lte": end_time
                }
            }
        }
    }
    res = es.search(index=elasticsearch_index, body=query)
    data = []
    for hit in res['hits']['hits']:
        data.append(hit['_source'])
    return data

fetched_data = get_data()
data = pd.DataFrame(fetched_data)

In [31]:
# TODO: remove duplicates coordinates, so clustering is more accurate

data = data.drop_duplicates(subset=['lat', 'lon'])
# print(data)

In [35]:

coordinates_data = data[['lat', 'lon']]
# print(coordinates_data)
kmeans = KMeans(n_clusters=1, init='k-means++', random_state=42)
clustered_data = kmeans.fit_predict(coordinates_data)

print(clustered_data)

# plt.scatter(clustered_data['value'], clustered_data['value'], c=clustered_data['cluster'], cmap='viridis')
# plt.xlabel('value')
# plt.ylabel('value')

# plt.show()


[0]
